In [3]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)
print(inputs.shape)

torch.Size([6, 3])


In [4]:
x_2=inputs[1]# our query is journey
d_in=inputs.shape[1]
d_out=2

In [5]:
torch.manual_seed(123)
w_q=torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
w_k=torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
w_v=torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)

In [6]:
print(w_q)

Parameter containing:
tensor([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]])


In [7]:
key_2=x_2 @ w_k
values_2=x_2 @ w_v
query_2=x_2 @ w_q


key=inputs @ w_k
values=inputs @ w_v
query=inputs @ w_q
print(key)


In [26]:
key=inputs @ w_k
values=inputs @ w_v
query=inputs @ w_q
print(key)
print(query.shape)


tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1827, 0.3292],
        [0.3275, 0.9642]])
torch.Size([6, 2])


In [28]:
# now we are focusing on the evaluating attention score
attention_score_2=query_2 @ key.T
print(attention_score_2)


tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [32]:
attention_scores=query @ key.T
attention_scores

tensor([[0.9231, 1.3545, 1.3241, 0.7910, 0.4032, 1.1330],
        [1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
        [1.2544, 1.8284, 1.7877, 1.0654, 0.5508, 1.5238],
        [0.6973, 1.0167, 0.9941, 0.5925, 0.3061, 0.8475],
        [0.6114, 0.8819, 0.8626, 0.5121, 0.2707, 0.7307],
        [0.8995, 1.3165, 1.2871, 0.7682, 0.3937, 1.0996]])

In [42]:
#first we need to normalise the value to get attention weight
d_k=key.shape[-1]
att_weight=torch.softmax((attention_scores/d_k**0.5),dim=-1)
att_weight


tensor([[0.1551, 0.2104, 0.2059, 0.1413, 0.1074, 0.1799],
        [0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820],
        [0.1503, 0.2256, 0.2192, 0.1315, 0.0914, 0.1819],
        [0.1591, 0.1994, 0.1962, 0.1477, 0.1206, 0.1769],
        [0.1610, 0.1949, 0.1923, 0.1501, 0.1265, 0.1752],
        [0.1557, 0.2092, 0.2048, 0.1419, 0.1089, 0.1794]])

In [ ]:
##WHY DIVIDE BY SQRT (DIMENSION)
Reason 1: For stability in learning

The softmax function is sensitive to the magnitudes of its inputs. When the inputs are large, the differences between the exponential values of each input become much more pronounced. This causes the softmax output to become "peaky," where the highest value receives almost all the probability mass, and the rest receive very little.

In attention mechanisms, particularly in transformers, if the dot products between query and key vectors become too large (like multiplying by 8 in this example), the attention scores can become very large. This results in a very sharp softmax distribution, making the model overly confident in one particular "key." Such sharp distributions can make learning unstable,

In [50]:
import torch 
inputs=torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])
softmax_result=torch.softmax(inputs,dim=-1)
print(f"normalised values{softmax_result}")
tensor_8=inputs*8
softmax_result=torch.softmax(tensor_8,dim=-1)
print(f"normalized values tensor_8 {softmax_result}")


normalised valuestensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])
normalized values tensor_8 tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])


In [ ]:
## BUT WHY SQRT?
Reason 2: To make the variance of the dot product stable

The dot product of Q and K increases the variance because multiplying two random numbers increases the variance.

The increase in variance grows with the dimension.

Dividing by sqrt (dimension) keeps the variance close to 1

In [52]:
import numpy as np

# Function to compute variance before and after scaling
def compute_variance(dim, num_trials=1000):
    dot_products = []
    scaled_dot_products = []

    # Generate multiple random vectors and compute dot products
    for _ in range(num_trials):
        q = np.random.randn(dim)
        k = np.random.randn(dim)
        
        # Compute dot product
        dot_product = np.dot(q, k)
        dot_products.append(dot_product)
        
        # Scale the dot product by sqrt(dim)
        scaled_dot_product = dot_product / np.sqrt(dim)
        scaled_dot_products.append(scaled_dot_product)
    
    # Calculate variance of the dot products
    variance_before_scaling = np.var(dot_products)
    variance_after_scaling = np.var(scaled_dot_products)

    return variance_before_scaling, variance_after_scaling

# For dimension 5
variance_before_5, variance_after_5 = compute_variance(5)
print(f"Variance before scaling (dim=5): {variance_before_5}")
print(f"Variance after scaling (dim=5): {variance_after_5}")

# For dimension 20
variance_before_100, variance_after_100 = compute_variance(100)
print(f"Variance before scaling (dim=100): {variance_before_100}")
print(f"Variance after scaling (dim=100): {variance_after_100}")



Variance before scaling (dim=5): 4.804038701221977
Variance after scaling (dim=5): 0.9608077402443952
Variance before scaling (dim=100): 103.77816201692286
Variance after scaling (dim=100): 1.0377816201692285


In [56]:
context_vector = att_weight @ values
context_vector

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]])

In [80]:
import torch
class self_attention_v1:
    def __init__(self,din,dout):
        self.din=din
        self.dout=dout
        self.w_q=torch.nn.Parameter(torch.rand(d_in,d_out))
        self.w_k=torch.nn.Parameter(torch.rand(d_in,d_out))
        self.w_v=torch.nn.Parameter(torch.rand(d_in,d_out))
    def forward(self,x):
        keys=x @ self.w_k
        values=x @self.w_v
        query=x @ self.w_q
        att_score=queries @ keys.T # omega
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        context_vec = attn_weights @ values
        return context_vec
        

In [82]:
torch.manual_seed(123)
sa_v1 =self_attention_v1(d_in, d_out)
print(sa_v1.forward(inputs))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x5 and 3x2)